In [4]:
import html
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch
import spacy
from textblob import TextBlob

# Load NLP model for content understanding (spaCy for basic NLP tasks)
nlp = spacy.load("en_core_web_sm")

# Load the model and tokenizer
model_name = 'google/flan-t5-small'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

def analyze_content_structure(text):
    """
    Analyze text to understand its structure, key elements, and suggest the best summarization format.
    """
    doc = nlp(text)
    num_entities = len(doc.ents)
    num_sentences = len(list(doc.sents))
    num_tokens = len(doc)
    num_keywords = sum(1 for token in doc if not token.is_stop)
    num_bullets = text.count('- ') + text.count('* ') + text.count('• ')
    num_arrows = text.count('->') + text.count('=>')

    # Analysis based on detected entities and structure
    if num_bullets > 3 or 'list' in text.lower():
        return "bullet points"
    elif num_arrows > 2 or "process" in text.lower() or "steps" in text.lower():
        return "flowchart"
    elif num_entities > 5 and num_keywords / num_tokens < 0.5:
        return "table"
    else:
        return "paragraph"

def get_optional_context():
    context = input("Would you like to provide additional context for better summarization? (optional, press enter to skip): ")
    return context

def analyze_sentiment(text):
    """
    Perform sentiment analysis on the given text.
    """
    blob = TextBlob(text)
    sentiment = blob.sentiment
    return sentiment.polarity, sentiment.subjectivity

def clean_text(text):
    """
    Clean the input text to remove unwanted HTML-like entities and tags.
    """
    # Decode any HTML-encoded entities
    text = html.unescape(text)
    # Remove any remaining HTML tags
    text = text.replace('&nbsp;', ' ').replace('&lt;', '<').replace('&gt;', '>')
    # Additional cleaning can be done if necessary
    return text

def clean_summary(summary, style):
    """
    Post-process the summary to clean up numbering and ensure proper formatting.
    """
    if style == "bullet points":
        cleaned_lines = []
        for line in summary.split(". "):
            line = line.strip().lstrip("1234567890.- ")
            if line:
                cleaned_lines.append(f"- {line}")
        return "\n".join(cleaned_lines)
    elif style == "flowchart":
        cleaned_lines = []
        for line in summary.split(". "):
            line = line.strip().lstrip("1234567890.- ")
            if line:
                cleaned_lines.append(f"-> {line}")
        return "\n".join(cleaned_lines)
    else:
        return summary.replace(". ", ".\n")

def summarize_section(section_text, context='', style='paragraph'):
    """
    Summarizes a single section of text.
    """
    # Clean the section text before summarizing
    section_text = clean_text(section_text)

    # Combine user context with the main text
    combined_input = context + "\n" + section_text if context else section_text

    # Create an adaptive prompt for the T5 model
    prompt = f"summarize the content as {style}: {combined_input}"

    # Calculate dynamic max and min lengths based on input length
    input_tokens = tokenizer.encode(prompt, return_tensors="pt", max_length=1024, truncation=True)
    input_length = input_tokens.shape[1]
    max_length = 5000 # 1/4 of input length, but at least 150
    min_length = 1000   # 1/6 of input length, but at least 40

    # Generate summary
    summary_ids = model.generate(
        input_tokens,
        max_length=max_length,
        min_length=min_length,
        length_penalty=2.0,
        num_beams=4,
        no_repeat_ngram_size=3,  # Prevents repeating phrases
        early_stopping=True
    )

    # Decode summary
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

def summarize_text(file_path, context=''):
    """
    Summarizes the text from the given file using the context and chosen style.
    """
    # Read the input file
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

    # Clean the entire text before splitting into sections
    text = clean_text(text)

    # Split the text by sections based on headings for more focused summarization
    sections = text.split("\n## ")
    summarized_sections = []

    for section in sections:
        # Determine summarization style for each section
        style = analyze_content_structure(section)
        # Summarize the section
        section_summary = summarize_section(section, context, style)
        # Clean the summarized output
        cleaned_summary = clean_summary(section_summary, style)
        summarized_sections.append(f"## {cleaned_summary}")

    # Join all summarized sections
    final_summary = "\n\n".join(summarized_sections)

    # Perform sentiment analysis on the original text
    polarity, subjectivity = analyze_sentiment(text)

    # Beautify the output
    output = f"\n{'='*40}\n"
    output += f"Summarized Text:\n{'-'*40}\n"
    output += final_summary + "\n"
    # output += f"\n{'='*40}\n"
    # output += f"Sentiment Analysis:\n{'-'*40}\n"
    # output += f"Polarity: {'Positive' if polarity > 0 else 'Negative' if polarity < 0 else 'Neutral'} ({polarity:.2f})\n"
    # output += f"Subjectivity: {'Subjective' if subjectivity > 0.5 else 'Objective'} ({subjectivity:.2f})\n"
    # output += f"{'='*40}\n"

    return output

def main():
    input_data = "/Users/adityachavan/Downloads/TEXTSUMM/ss.txt"
    context = get_optional_context()

    # Generate the summary
    summarized_text = summarize_text(input_data, context)

    # Display the summarized text
    print(summarized_text)

# Run the main function
if __name__ == "__main__":
    main()



Summarized Text:
----------------------------------------
## -> Artificial Intelligence (AI) is transforming industries across the globe by introducing advanced capabilities that automate tasks, enhance decision-making, and drive e>iciency
-> From healthcare to finance, AI technologies are being used in a variety of sectors to solve complex problems and deliver innovative solutions
-> From the mid-20th century, researchers began exploring the possibility of creating machines that could think and learn
-> Early developments in AI focused on rule- based systems, where computers followed predefined rules to achieve specific tasks
-> However, these systems were limited in scope and could only handle simple, structured problems
-> With the advent of machine learning in the 21st century, AI experienced a significant leap forward
-> Machine learning allows computers to learn from data without being explicitly programmed, enabling them to handle more complex and unstructured tasks
-> AI-drive